In [48]:
from google.colab import files
files.upload()

KeyboardInterrupt: ignored

In [0]:
!ls -lha kaggle.json

In [0]:
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle competitions download -c ashrae-energy-prediction

In [0]:
!unzip 'train.csv.zip'
!unzip 'test.csv.zip'
!unzip 'weather_test.csv.zip'
!unzip 'weather_train.csv.zip'
!unzip 'sample_submission.csv.zip'

In [0]:
!ls

In [0]:
!pip install meteocalc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm_notebook as tqdm
import datetime
from meteocalc import feels_like, Temp
from sklearn import metrics
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Load Data

In [0]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [0]:
train_df = pd.read_csv('train.csv')
building_df = pd.read_csv('building_metadata.csv')
weather_df = pd.read_csv('weather_train.csv')

In [51]:
weather_df.head(20)

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6
5,0,2016-01-01 05:00:00,19.4,NaN,19.4,0.0,NaN,0.0,0.0
6,0,2016-01-01 06:00:00,21.1,6.0,21.1,-1.0,1019.4,0.0,0.0
7,0,2016-01-01 07:00:00,21.1,NaN,21.1,0.0,1018.8,210.0,1.5
8,0,2016-01-01 08:00:00,20.6,NaN,20.0,0.0,1018.1,0.0,0.0
9,0,2016-01-01 09:00:00,21.1,NaN,20.6,0.0,1019.0,290.0,1.5


Referring to the following discussions. We have filtered the data in the next step

* Ref1. https://www.kaggle.com/c/ashrae-energy-prediction/discussion/114830#latest-680086
* Ref2. https://www.kaggle.com/c/ashrae-energy-prediction/discussion/113054#656588 

In [0]:
# Original code from https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude
def fill_weather_data(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week','month'],axis=1)
    
    def get_meteorological_features(data):
        def calculate_rh(df):
            df['relative_humidity'] = 100 * (np.exp((17.625 * df['dew_temperature']) / (243.04 + df['dew_temperature'])) / np.exp((17.625 * df['air_temperature'])/(243.04 + df['air_temperature'])))
        def calculate_fl(df):
            flike_final = []
            flike = []
            # calculate Feels Like temperature
            for i in range(len(df)):
                at = df['air_temperature'][i]
                rh = df['relative_humidity'][i]
                ws = df['wind_speed'][i]
                flike.append(feels_like(Temp(at, unit = 'C'), rh, ws))
            for i in range(len(flike)):
                flike_final.append(flike[i].f)
            df['feels_like'] = flike_final
            del flike_final, flike, at, rh, ws
        calculate_rh(data)
        calculate_fl(data)
        return data

    weather_df = get_meteorological_features(weather_df)
    return weather_df

In [0]:
def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekday"] = df["timestamp"].dt.weekday
    df["dayofweek"] = df["timestamp"].dt.dayofweek
    
    df['month'] = df['timestamp'].dt.month
    df['month'].replace((1, 2, 3), 1, inplace = True)
    df['month'].replace((4, 5, 6,), 2, inplace = True)
    df['month'].replace((7,8,9), 3, inplace = True)
    df['month'].replace((10,11,12), 4, inplace = True)
  
    df['square_feet'] =  np.log1p(df['square_feet'])
    
    # Remove Unused Columns
    drop = ["timestamp"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [54]:
weather_df=fill_weather_data(weather_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [55]:
weather_df.head(20)

,site_id,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,timestamp,wind_direction,wind_speed,relative_humidity,feels_like
0,0,25.0,6.000000,20.0,-0.173913,1019.700000,2016-01-01 00:00:00,0.0,0.0,73.802512,77.000000
1,0,24.4,4.285714,21.1,-1.000000,1020.200000,2016-01-01 01:00:00,70.0,1.5,81.864391,75.920000
2,0,22.8,2.000000,21.1,0.000000,1020.200000,2016-01-01 02:00:00,0.0,0.0,90.149085,73.040000
3,0,21.1,2.000000,20.6,0.000000,1020.100000,2016-01-01 03:00:00,0.0,0.0,96.971205,69.980000
4,0,20.0,2.000000,20.0,-1.000000,1020.000000,2016-01-01 04:00:00,250.0,2.6,100.000000,68.000000
5,0,19.4,4.285714,19.4,0.000000,1018.926087,2016-01-01 05:00:00,0.0,0.0,100.000000,66.920000
6,0,21.1,6.000000,21.1,-1.000000,1019.400000,2016-01-01 06:00:00,0.0,0.0,100.000000,69.980000
7,0,21.1,4.285714,21.1,0.000000,1018.800000,2016-01-01 07:00:00,210.0,1.5,100.000000,69.980000
8,0,20.6,4.285714,20.0,0.000000,1018.100000,2016-01-01 08:00:00,0.0,0.0,96.361670,69.080000
9,0,21.1,4.285714,20.6,0.000000,1019.000000,2016-01-01 09:00:00,290.0,1.5,96.971205,69.980000


In [56]:
# memory reduction
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16=True)
weather_df = reduce_mem_usage(weather_df,use_float16=True)

# merge data
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])
del weather_df
gc.collect()

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 11.80 MB
Memory usage after optimization is: 3.19 MB
Decreased by 72.9%


11

In [0]:
# feature engineering
train_df = features_engineering(train_df)

# transform target variable
train_df['meter_reading'] = np.log1p(train_df["meter_reading"])

In [58]:
train_df.head(20)

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,relative_humidity,feels_like,hour,weekday,dayofweek,month
0,0,0,0.0,0,0,8.913685,2008.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
1,1,0,0.0,0,0,7.908755,2004.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
2,2,0,0.0,0,0,8.589886,1991.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
3,3,0,0.0,0,0,10.072639,2002.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
4,4,0,0.0,0,0,11.666573,1975.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
5,5,0,0.0,0,0,8.987322,2000.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
6,6,0,0.0,0,4,10.237349,1981.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
7,7,0,0.0,0,0,11.704165,1989.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
8,8,0,0.0,0,0,11.015510,2003.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1
9,9,0,0.0,0,6,10.203629,2010.0,NaN,25.0,6.0,20.0,-0.17395,1019.5,0.0,0.0,73.8125,77.0,0,4,4,1


In [0]:
# declare target, categorical and numeric columns
target = 'meter_reading'
categorical = ['building_id', 'site_id', 'primary_use', 'meter','dayofweek','weekday','month']
numeric_cols = [col for col in train_df.columns if col not in categorical + [target, 'timestamp']]
features = categorical + numeric_cols

In [0]:
import seaborn as sns
def run_lgbm(train, cat_features = categorical, num_rounds = 20000, folds = 2):
    kf = StratifiedKFold(n_splits=folds, shuffle=False, random_state=2319)
    models = []
    feature_importance_df = pd.DataFrame()

    param =  {'num_leaves': 1580,
             'objective': 'regression',
             'learning_rate': 0.2,
             'boosting': 'gbdt',
             'subsample': 0.4,
             'feature_fraction': 0.7,
             'n_jobs': -1,
             'seed': 50,
             'metric': 'rmse'
              }
    preonTrain = np.zeros(len(train))
  
    for tr_idx, val_idx in tqdm(kf.split(train_df,train_df['month']), total = folds):
        tr_x, tr_y = train[features].iloc[tr_idx], train[target].iloc[tr_idx]
        vl_x, vl_y = train[features].iloc[val_idx], train[target].iloc[val_idx]
        tr_data = lgb.Dataset(tr_x, label = tr_y,  categorical_feature = categorical)
        vl_data = lgb.Dataset(vl_x, label = vl_y,  categorical_feature = categorical)
        clf = lgb.train(param, tr_data, num_rounds, valid_sets = [tr_data, vl_data], verbose_eval = 25, 
                        early_stopping_rounds = 50)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = features
        fold_importance_df["importance"] = clf.feature_importance()
        
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        models.append(clf)
        predonTrain[val_idx] = clf.predict(vl_x)
        gc.collect()
    score = np.sqrt(metrics.mean_squared_error(train[target], np.clip(predonTrain, a_min=0, a_max=None)))
    print('Our score on training set is :', score)
    
    cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:20].index)
    best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

    plt.figure(figsize=(14,26))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance",ascending=False))
    plt.title('LightGBM Features (averaged over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances.png')

    return models
models = run_lgbm(train_df)

In [61]:
# read test
test_df = pd.read_csv('test.csv')
row_ids = test_df["row_id"]
test_df.drop("row_id", axis=1, inplace=True)
test_df = reduce_mem_usage(test_df)

# merge with building info
test_df = test_df.merge(building_df,left_on='building_id',right_on='building_id',how='left')
del building_df
gc.collect()

# fill test weather data
weather_df = pd.read_csv('weather_test.csv')


Memory usage of dataframe is 954.38 MB
Memory usage after optimization is: 199.59 MB
Decreased by 79.1%


In [62]:
weather_df = fill_weather_data(weather_df)
weather_df = reduce_mem_usage(weather_df)

# merge weather data
test_df = test_df.merge(weather_df,how='left',on=['timestamp','site_id'])
del weather_df
gc.collect()

# feature engineering
test_df = features_engineering(test_df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


Memory usage of dataframe is 23.53 MB
Memory usage after optimization is: 11.18 MB
Decreased by 52.5%


In [63]:
def predictions(models, iterations = 120):
    # split test data into batches
    set_size = len(test_df)
    batch_size = set_size // iterations
    meter_reading = []
    for i in tqdm(range(iterations)):
        pos = i*batch_size
        fold_preds = [np.expm1(model.predict(test_df[features].iloc[pos : pos+batch_size])) for model in models]
        meter_reading.extend(np.mean(fold_preds, axis=0))

    print(len(meter_reading))
    assert len(meter_reading) == set_size
    submission = pd.read_csv('sample_submission.csv')
    submission['meter_reading'] = np.clip(meter_reading, a_min=0, a_max=None) # clip min at zero
    submission.to_csv('fe2_lgbm.csv', index=False)
    print('We are done!')
predictions(models)


41697600
We are done!


In [0]:
l=pd.read_csv('fe2_lgbm.csv')

In [65]:
l.head(50)

,row_id,meter_reading
0,0,0.566554
1,1,0.634895
2,2,0.329115
3,3,0.520048
4,4,0.511409
5,5,0.361532
6,6,0.679883
7,7,0.721906
8,8,106.048052
9,9,0.504067


In [0]:
!kaggle competitions submit -c ashrae-energy-prediction -f 'fe2_lgbm.csv' -m "LGBM Improve4"

In [67]:
del l
gc.collect()

475